In [1]:
import sys
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from analyzer import Analyzer

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams["font.size"] = 16

In [2]:
filein="/workspace/HDD2/pharm/shionogi/voriconazole/raw/E0263 1103 - 2022-10-07 13.17.51.ndpi"
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda', index=0)

In [3]:
!nvidia-smi

Sun Dec 15 15:33:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   34C    P8              23W / 300W |     26MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
dat=Analyzer(DEVICE=DEVICE)

In [5]:
%%time
dat.load_model(
    dir_featurize_model="/workspace/pathology/models/fold/bt0/model_ssl.pt",
    dir_classification_models="/workspace/pathology/models/analyzer/bt_layer45_pu.pickle",
    style="dict"
)

/workspace/pathology/src/SelfSupervisedLearningPathology/analyzer/model.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(dir_model))


CPU times: user 576 ms, sys: 119 ms, total: 696 ms
Wall time: 907 ms


In [6]:
%%time
mask=dat.ImageProcessor.get_mask_inside(
    filein=filein,
    patch_size=448,
    slice_min_patch=100,
)

CPU times: user 45.6 ms, sys: 7.17 ms, total: 52.7 ms
Wall time: 51.5 ms


In [7]:
%%time
locations=[
    dat.ImageProcessor.get_locations(
        filein=filein, 
        mask=mask, 
        patch_size=448,
        model_patch_size=224
    ),# small size
    dat.ImageProcessor.get_locations(
        filein=filein, 
        mask=mask, 
        patch_size=448,
        model_patch_size=448
    ),# large size
]

CPU times: user 24.2 ms, sys: 1.11 ms, total: 25.3 ms
Wall time: 24.6 ms


In [9]:
%%time
from analyzer import prepare_dataset_location
data_loaders=[
    prepare_dataset_location(
        filein=filein,
        locations=locations[0], 
        batch_size=256,
        patch_size=224
    ),
    prepare_dataset_location(
        filein=filein,
        locations=locations[1], 
        batch_size=256,
        patch_size=448
    ),            
]

CPU times: user 19.4 ms, sys: 7.81 ms, total: 27.2 ms
Wall time: 26 ms


In [10]:
%%time
result_patch, result_all=dat.FindingClassifier.classify(
    data_loaders, num_pool=int(448/224)**2
)

Featurizing WSI


Finding Classsifying


100%|██████████| 6/6 [00:00<00:00, 294.76it/s]

CPU times: user 27.7 s, sys: 4.43 s, total: 32.2 s
Wall time: 3min 13s


In [11]:
result_all

{'Hepatocellular Degeneration': array([0.33314529]),
 'Hepatocellular Injury, and Death': array([0.73333333]),
 'Billary Change': array([0.26662946]),
 'Hepatocellular Responses': array([6.46878339e-21]),
 'Inflammation': array([0.4]),
 'Proliferative Lesions': array([0.99999987])}